# Timing Model useage example

## Build a timing model 

In [1]:
from pint.models import get_model

One can build a timing model via get_model() method. get_model() will make the model according to the .par file. All the model components, delays or phases, will be stored in order. 

In [2]:
par = 'B1855+09_NANOGrav_dfg+12_TAI.par'
m = get_model(par)


In [3]:
print m.F0.quantity
print m.F0.units
print m.F0.description

186.494081567 Hz
Hz
Spin-frequency


In [4]:
m.dispersion_delay

<bound method DispersionDMX.dispersion_delay of <pint.models.dispersion_model.DispersionDMX object at 0x1a183eb9d0>>

To take a look what type of model component are in the model

In [5]:
m.component_types # We have delay component and phase component 

['DelayComponent', 'PhaseComponent']

TimingModel class stores the delay model components and phase components in the lists 

In [6]:
m.DelayComponent_list # Delays are stored in the DelayComponent_list

In [7]:
m.PhaseComponent_list # phases are stored in the PhaseComponent_list

To add a component:

In [8]:
from pint.models.astrometry import AstrometryEcliptic

In [9]:
a = AstrometryEcliptic() # init the AstrometryEcliptic instance

Add the new component instance into time model with order 3

In [10]:
m.add_component(a, 3)

In [11]:
m.DelayComponent_list # The new instance is added to delay component list 
                      # index 3

To remove a component is simple. Just use remove it from the list. You can map the component instance using name string via map_component() method

In [12]:
component, order, from_list, comp_type = \
    m.map_component('AstrometryEcliptic')
from_list.remove(component)

In [13]:
m.DelayComponent_list # AstrometryEcliptic is removed from delay list. 

To switch the order of a component, just change the index in the component list. 

In [16]:
# First map the component instance
component, order, from_list, comp_type = \
    m.map_component('PhaseJump')
# If one wants to move this component to a new order without swapping
from_list.remove(component)
from_list.insert(5, component)

In [17]:
m.DelayComponent_list

In [21]:
# If one wants to swap with other component
component, order, from_list, comp_type = \
    m.map_component('DispersionDMX')
new_order = 3
from_list[order], from_list[new_order] = from_list[new_order], from_list[order]

In [22]:
m.DelayComponent_list

Delays will be computed in order. 

In [23]:
# First get the toas
from pint.toa import get_TOAs
t = get_TOAs('../tests/datafile/B1855+09_NANOGrav_dfg+12.tim')

INFO: Applying clock corrections (include_GPS = True, include_BIPM = True. [pint.toa]
INFO: Observatory arecibo, loading clock file /Users/jingluo/.local/lib/python2.7/site-packages/pint-0.5.3+201.gd7af879.dirty-py2.7-macosx-10.7-x86_64.egg/pint/datafiles/time.dat [pint.observatory.topo_obs]
INFO: Evaluating observatory clock corrections. [pint.observatory.topo_obs]
INFO: Applying GPS to UTC clock correction (~few nanoseconds) [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading GPS clock file /Users/jingluo/.local/lib/python2.7/site-packages/pint-0.5.3+201.gd7af879.dirty-py2.7-macosx-10.7-x86_64.egg/pint/datafiles/gps2utc.clk [pint.observatory.topo_obs]
INFO: Applying TT(TAI) to TT(BIPM) clock correction (~27 us) [pint.observatory.topo_obs]
INFO: Observatory arecibo, loading BIPM clock file /Users/jingluo/.local/lib/python2.7/site-packages/pint-0.5.3+201.gd7af879.dirty-py2.7-macosx-10.7-x86_64.egg/pint/datafiles/tai2tt_bipm2015.clk [pint.observatory.topo_obs]
INFO: Computing

In [24]:
# compute the total delay
total_delay = m.delay(t)
total_delay

<Quantity [ 386.46306501, 292.74493628, 123.64677909, -68.30001643,
           -391.49294556,-391.55211107,-317.02034107,-317.01903715,
           -317.01690251,-317.01021663,-172.22028083, 240.44391033,
            240.52758075, 366.04797969, 416.44705584, 416.38194674,
            238.67530959,-141.66866147,-368.26117845, 188.93718598,
            409.62695507, 360.61475497, 276.16176165, -56.48086239,
           -403.64873011, 401.8358864 , 373.61321114,   9.65651057,
           -325.51347386,-401.49974263, 386.45719164, 292.73906189,
            123.64090449, -68.30589104,-391.49881966,-391.55798517,
           -317.02621416,-317.02491024,-317.0227756 ,-317.01608972,
           -172.22615364, 240.43803739, 240.52170781, 366.0421064 ,
            416.44118193, 416.37607283, 238.66943504,-141.67453614,
           -368.26705281,-434.64202751,-318.54915238, -59.74843801,
            188.93131282, 409.62108124, 360.60888042, 143.43395202,
            415.55294927, 276.1558868 , -56.4867

One can get the delay upto some component. For example, I want to the delay computation stop at jump delay.

In [26]:
to_jump_delay = m.delay(t, cutoff_component='SolarSystemShapiro')
to_jump_delay

<Quantity [ 392.26833915, 299.04792455, 129.09853302, -77.82209388,
           -383.28053432,-383.31863655,-321.21094754,-321.19836098,
           -321.17772364,-321.11284022,-170.56138387, 231.01083192,
            231.08567389, 369.80791309, 410.83873507, 410.82464089,
            229.38415611,-137.72845021,-360.02656371, 180.05446962,
            401.6445299 , 367.08587606, 284.93857219, -48.56764114,
           -396.77607623, 394.80676361, 379.06111526,  18.21757288,
           -335.02874831,-395.87680334, 392.26833914, 299.04792458,
            129.09853306, -77.82209383,-383.2805343 ,-383.31863653,
           -321.21094757,-321.19836101,-321.17772367,-321.11284025,
           -170.5613839 , 231.01083188, 231.08567386, 369.80791307,
            410.83873507, 410.8246409 , 229.38415615,-137.72845017,
           -360.02656368,-427.70216108,-321.29313448, -67.18876902,
            180.05446958, 401.64452989, 367.08587608, 141.24499898,
            416.64284814, 284.93857222, -48.5676

In [27]:
m.F1